In [2]:
using SymPy

# 4.5

図4.26よりDHパラは↓こんなカンジ

|  $i$  |  $a_i$   | $\alpha_i$ | $d_i$ | $\theta_i$|
| --- | --- | -------- | -------- | -------- |
|   1  |  0   |   0       |      0    |    $\theta_1$ |
|   2  |  $L_a$   |  0    |     0     |    $\theta_2$      |
|   3  |  $L_b$   |   $\pi$       |     $d_3$     |     0     |
|    4 |  0   |  0     |  0    | $\theta_4$   |

In [3]:
@vars θ₁ θ₂ d₃ θ₄ Lₐ Lᵦ  

(θ₁, θ₂, d₃, θ₄, Lₐ, Lᵦ)

In [4]:
_⁰T₁=Sym[cos(θ₁) -sin(θ₁) 0 0;
     sin(θ₁) cos(θ₁)  0 0;
    0        0       1  0;
    0        0       0  1]

4×4 Matrix{Sym}:
 cos(θ₁)  -sin(θ₁)  0  0
 sin(θ₁)   cos(θ₁)  0  0
       0         0  1  0
       0         0  0  1

In [5]:
_¹T₂=Sym[cos(θ₂) -sin(θ₂) 0 Lₐ;
     sin(θ₂) cos(θ₂)  0 0;
    0        0       1  0;
    0        0       0  1]

4×4 Matrix{Sym}:
 cos(θ₂)  -sin(θ₂)  0  Lₐ
 sin(θ₂)   cos(θ₂)  0   0
       0         0  1   0
       0         0  0   1

In [6]:
_²T₃=Sym[1 0 0 Lᵦ;
     0 -1  0 0;
    0  0 -1  -d₃;
    0  0  0  1]

4×4 Matrix{Sym}:
 1   0   0   Lᵦ
 0  -1   0    0
 0   0  -1  -d₃
 0   0   0    1

In [7]:
_³T₄=Sym[cos(θ₄) -sin(θ₄) 0 0;
     sin(θ₄) cos(θ₄)  0 0;
    0        0       1  0;
    0        0       0  1]

4×4 Matrix{Sym}:
 cos(θ₄)  -sin(θ₄)  0  0
 sin(θ₄)   cos(θ₄)  0  0
       0         0  1  0
       0         0  0  1

In [10]:
_⁰T₄ = _⁰T₁ * _¹T₂ *_²T₃ * _³T₄
display(sympy.trigsimp((sympy.factor(_⁰T₄))))

⎡cos(θ₁ + θ₂ - θ₄)  sin(θ₁ + θ₂ - θ₄)   0   Lᵦ⋅cos(θ₁ + θ₂) + Lₐ⋅cos(θ₁)⎤
⎢                                                                       ⎥
⎢sin(θ₁ + θ₂ - θ₄)  -cos(θ₁ + θ₂ - θ₄)  0   Lᵦ⋅sin(θ₁ + θ₂) + Lₐ⋅sin(θ₁)⎥
⎢                                                                       ⎥
⎢        0                  0           -1              -d₃             ⎥
⎢                                                                       ⎥
⎣        0                  0           0                1              ⎦

# 4.6

$^0T_4$より$(x,y,z)^T$は

In [12]:
sympy.trigsimp((sympy.factor(_⁰T₄[1:3,4])))

⎡Lᵦ⋅cos(θ₁ + θ₂) + Lₐ⋅cos(θ₁)⎤
⎢                            ⎥
⎢Lᵦ⋅sin(θ₁ + θ₂) + Lₐ⋅sin(θ₁)⎥
⎢                            ⎥
⎣            -d₃             ⎦

加えてこのロボットでは姿勢を１軸制御できてそれは図より
$\theta_1 + \theta_2 - \theta_4$
これらを$q$としてまとめる

In [14]:
q = [_⁰T₄[1:3,4]; θ₁+θ₂-θ₄]
sympy.trigsimp((sympy.factor(q)))

⎡Lᵦ⋅cos(θ₁ + θ₂) + Lₐ⋅cos(θ₁)⎤
⎢                            ⎥
⎢Lᵦ⋅sin(θ₁ + θ₂) + Lₐ⋅sin(θ₁)⎥
⎢                            ⎥
⎢            -d₃             ⎥
⎢                            ⎥
⎣        θ₁ + θ₂ - θ₄        ⎦

するとヤコビアンJは

In [15]:
J = Sym[diff.(q, θ₁) diff.(q, θ₂) diff.(q, d₃) diff.(q, θ₄)]
J = sympy.trigsimp((sympy.factor(J)))

⎡-Lᵦ⋅sin(θ₁ + θ₂) - Lₐ⋅sin(θ₁)  -Lᵦ⋅sin(θ₁ + θ₂)  0   0 ⎤
⎢                                                       ⎥
⎢Lᵦ⋅cos(θ₁ + θ₂) + Lₐ⋅cos(θ₁)   Lᵦ⋅cos(θ₁ + θ₂)   0   0 ⎥
⎢                                                       ⎥
⎢              0                       0          -1  0 ⎥
⎢                                                       ⎥
⎣              1                       1          0   -1⎦

## 補足
ある姿勢をとったときのヤコビアンをsympyで求めるには
- 例
    - $q = [\pi/2~~\pi/4~~0.1~-\pi/4]^T$, $L_a=0.5, L_b=0.3$のとき

In [16]:
J_act = J.subs(Lₐ, 0.5).subs(Lᵦ, 0.3).subs(θ₁, pi/2).subs(θ₂, pi/4).subs(d₃, 0.1).subs(θ₄, -pi/4)
#一気に渡す方法がありそう

⎡-0.712132034355964  -0.212132034355964  0   0 ⎤
⎢                                              ⎥
⎢-0.212132034355964  -0.212132034355964  0   0 ⎥
⎢                                              ⎥
⎢        0                   0           -1  0 ⎥
⎢                                              ⎥
⎣        1                   1           0   -1⎦

- この行列は同じsymmatrix型の行列とは行列計算ができるが、普通のArray型とは計算できない？

In [30]:
J_act*SymPy.SymMatrix([1 1 1 1])'

⎡-0.924264068711929⎤
⎢                  ⎥
⎢-0.424264068711928⎥
⎢                  ⎥
⎢        -1        ⎥
⎢                  ⎥
⎣        1         ⎦

In [34]:
J_act*[1 1 1 1]'

4×1 Matrix{SymMatrix}:
 SymMatrix(PyObject Matrix([
[-0.712132034355964, -0.212132034355964,  0,  0],
[-0.212132034355964, -0.212132034355964,  0,  0],
[                 0,                  0, -1,  0],
[                 1,                  1,  0, -1]]))
 SymMatrix(PyObject Matrix([
[-0.712132034355964, -0.212132034355964,  0,  0],
[-0.212132034355964, -0.212132034355964,  0,  0],
[                 0,                  0, -1,  0],
[                 1,                  1,  0, -1]]))
 SymMatrix(PyObject Matrix([
[-0.712132034355964, -0.212132034355964,  0,  0],
[-0.212132034355964, -0.212132034355964,  0,  0],
[                 0,                  0, -1,  0],
[                 1,                  1,  0, -1]]))
 SymMatrix(PyObject Matrix([
[-0.712132034355964, -0.212132034355964,  0,  0],
[-0.212132034355964, -0.212132034355964,  0,  0],
[                 0,                  0, -1,  0],
[                 1,                  1,  0, -1]]))

## もうちょい考える
6軸ロボットの場合
例えばこんな↓

| $i$ | $a_i$ | $\alpha_i$ | $d_i$ | $\theta_i$ |
| --- | ----- | ---------- | ----- | ---------- |
| 1   | 0     | $\pi/2$       | $d_1$    | $\theta_1$ |
| 2   | $-a_2$ | 0          | 0     | $\theta_2$ |
| 3   | $-a_3$ | 0      | 0 | $\theta_3$         |
|  4   |    0   |     $\pi/2$         | $d_4$    |  $\theta_4$          |
|   5  |    0   |  $-\pi/2$          |    $d_5$  |  $\theta_5$          |
| 6   | 0     | 0    | $d_6$     | $\theta_6$ |

In [37]:
@vars θ₁ θ₂ θ₃ θ₄ θ₅ θ₆ d₁ a₂ a₃ d₄ d₅ d₆

(θ₁, θ₂, θ₃, θ₄, θ₅, θ₆, d₁, a₂, a₃, d₄, d₅, d₆)

In [39]:
_⁰T₁=Sym[cos(θ₁) -sin(θ₁)  0  0;
         0          0     -1 -d₁;
         sin(θ₁)   cos(θ₁) 0  0;
         0         0       0  1]

4×4 Matrix{Sym}:
 cos(θ₁)  -sin(θ₁)   0    0
       0         0  -1  -d₁
 sin(θ₁)   cos(θ₁)   0    0
       0         0   0    1

In [41]:
_¹T₂ =Sym[cos(θ₂) -sin(θ₂)  0  -a₂;
         sin(θ₂)   cos(θ₂)  0   0;
         0          0       1   0;
         0          0       0   1]

4×4 Matrix{Sym}:
 cos(θ₂)  -sin(θ₂)  0  -a₂
 sin(θ₂)   cos(θ₂)  0    0
       0         0  1    0
       0         0  0    1

In [42]:
_²T₃ = Sym[cos(θ₃) -sin(θ₃)  0  -a₃;
         sin(θ₃)   cos(θ₃)  0   0;
         0          0       1   0;
         0          0       0   1]

4×4 Matrix{Sym}:
 cos(θ₃)  -sin(θ₃)  0  -a₃
 sin(θ₃)   cos(θ₃)  0    0
       0         0  1    0
       0         0  0    1

In [43]:
_³T₄ =Sym[cos(θ₄) -sin(θ₄)  0  0;
         0          0     -1 -d₄;
         sin(θ₄)   cos(θ₄) 0  0;
         0         0       0  1]

4×4 Matrix{Sym}:
 cos(θ₄)  -sin(θ₄)   0    0
       0         0  -1  -d₄
 sin(θ₄)   cos(θ₄)   0    0
       0         0   0    1

In [45]:
_⁴T₅ = Sym[cos(θ₅) -sin(θ₅)  0  0;
         0          0      1   d₅;
         -sin(θ₅)   -cos(θ₅) 0  0;
         0         0       0  1]

4×4 Matrix{Sym}:
  cos(θ₅)  -sin(θ₅)  0   0
        0         0  1  d₅
 -sin(θ₅)  -cos(θ₅)  0   0
        0         0  0   1

In [48]:
_⁵T₆ = Sym[cos(θ₆) -sin(θ₆)  0   0;
         sin(θ₆)   cos(θ₆)  0   0;
         0          0       1   d₆;
         0          0       0   1]

4×4 Matrix{Sym}:
 cos(θ₆)  -sin(θ₆)  0   0
 sin(θ₆)   cos(θ₆)  0   0
       0         0  1  d₆
       0         0  0   1

In [50]:
_⁰T₆ = _⁰T₁ * _¹T₂ * _²T₃ * _³T₄ * _⁴T₅ * _⁵T₆
_⁰T₆ = sympy.trigsimp((sympy.factor(_⁰T₆)))

⎡  cos(θ₁ + θ₂ + θ₃ - θ₅ - θ₆)   cos(θ₁ + θ₂ + θ₃ + θ₅ + θ₆)   cos(θ₁ + θ₂ + θ
⎢- ─────────────────────────── + ─────────────────────────── + ───────────────
⎢               2                             2                               
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                sin(θ₅ + θ₆)⋅
⎢                                                                             
⎢                                                                             
⎣                                                                             

₃ - θ₄ - θ₅ - θ₆)   cos(θ₁ + θ₂ + θ₃ - θ₄ + θ₅ + θ₆)   cos(θ₁ + θ₂ + θ₃ + θ₄ -
───────────────── + ───────────────────────────────